In [3]:
import pandas as pd

# Load the customer prediction dataset
customer_df = pd.read_csv("customer_clv_with_predictions.csv")

# Load Trustpilot reviews
reviews_df = pd.read_csv("tmobile_reviews.csv")  # Ensure this file has 'review_text' column

In [4]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

# Apply sentiment analysis
def analyze_sentiment(text):
    score = analyzer.polarity_scores(str(text))  # ensure text is not NaN
    return score['compound']

# Create sentiment score and label
reviews_df['SentimentScore'] = reviews_df['review_text'].apply(analyze_sentiment)

def get_label(score):
    if score >= 0.05:
        return "Positive"
    elif score <= -0.05:
        return "Negative"
    else:
        return "Neutral"

reviews_df['SentimentLabel'] = reviews_df['SentimentScore'].apply(get_label)

In [5]:
# Merge by index — aligns the first review with the first customer
customer_df['SentimentScore'] = reviews_df['SentimentScore'].reindex(customer_df.index)
customer_df['SentimentLabel'] = reviews_df['SentimentLabel'].reindex(customer_df.index)

# Optional: fill missing values
customer_df['SentimentScore'].fillna(0.0, inplace=True)
customer_df['SentimentLabel'].fillna('Neutral', inplace=True)

C:\Users\Acer\AppData\Local\Temp\ipykernel_4532\1845984529.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  customer_df['SentimentScore'].fillna(0.0, inplace=True)
C:\Users\Acer\AppData\Local\Temp\ipykernel_4532\1845984529.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For 

In [6]:
sentiment_dummies = pd.get_dummies(customer_df['SentimentLabel'], prefix='Sentiment')
customer_df = pd.concat([customer_df, sentiment_dummies], axis=1)

# Optional: drop text label
customer_df.drop('SentimentLabel', axis=1, inplace=True)

In [7]:
customer_df.to_csv("customer_clv_with_sentiment.csv", index=False)
print("✅ Saved as customer_clv_with_sentiment.csv")

✅ Saved as customer_clv_with_sentiment.csv
